In [1]:
from functools import partial

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    # Locations in block unit
    _locations = \
            [(30,40), # depot
             (37,52), (49,49), # locations to visit
             (52,64), (20,26),
             (40,30), (21,47),
             (17,63), (31,62),
             (52,33), (51,21),
             (42,41), (31,32),
             ( 5,25), (12,42),
             (36,16), (52,41),
             (27,23), (17,33),
             (13,13), (57,58),
             (62,42), (42,57),
             (16,57), ( 8,52),
             ( 7,38), (27,68),
             (30,48), (43,67),
             (58,48), (58,27),
             (37,69), (38,46),
             (46,10), (61,33),
             (62,63), (63,69),
             (32,22), (45,35),
             (59,15), ( 5, 6),
             (10,17), (21,10),
             ( 5,64), (30,15),
             (39,10), (32,39),
             (25,32), (25,55),
             (48,28), (56,37)]
    
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe 'NY Times: Know Your distance'
    data['locations'] = [(l[0] * 114, l[1] * 80) for l in _locations]
    data['num_locations'] = len(data['locations'])
    data['demands'] = \
          [0 ,7, 30, 16, 9, 21, 15, 19, 23, 11, 5, 19, 29, 23, 21, 
                       10, 15, 3, 41, 9, 28, 8, 8, 16, 10, 28, 7, 15, 14, 6, 
                       19, 11, 12, 23, 26, 17, 6, 9, 15, 14, 7, 27, 13, 11, 16, 10, 
                       5, 25, 17, 18, 10]
    data['num_vehicles'] = 5
    data['vehicle_capacity'] = 160
    data['depot'] = 0
    return data


#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (
        abs(position_1[0] - position_2[0]) + abs(position_1[1] - position_2[1]))


def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            else:
                _distances[from_node][to_node] = (manhattan_distance(
                    data['locations'][from_node], data['locations'][to_node]))

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]
    
    return distance_evaluator


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(node)]

    return demand_evaluator


def add_capacity_constraints(routing, data, demand_evaluator_index):
    """Adds capacity constraint"""
    capacity = 'Capacity'
    routing.AddDimension(
        demand_evaluator_index,
        0,  # null capacity slack
        data['vehicle_capacity'],
        True,  # start cumul to zero
        capacity)


###########
# Printer #
###########
def print_solution(data, routing, manager, assignment):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print('Objective: {}'.format(assignment.ObjectiveValue()))
    total_distance = 0
    total_load = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            plan_output += ' {} Load({}) -> '.format(
                manager.IndexToNode(index), assignment.Value(load_var))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
        load_var = capacity_dimension.CumulVar(index)
        plan_output += ' {0} Load({1})\n'.format(
            manager.IndexToNode(index), assignment.Value(load_var))
        plan_output += 'Distance of the route: {}m\n'.format(distance)
        plan_output += 'Load of the route: {}\n'.format(
            assignment.Value(load_var))
        print(plan_output)
        total_distance += distance
        total_load += assignment.Value(load_var)
    print('Total Distance of all routes: {}m'.format(total_distance))
    print('Total Load of all routes: {}'.format(total_load))


########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define weight of each edge
    distance_evaluator = routing.RegisterTransitCallback(
        partial(create_distance_evaluator(data), manager))
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator)

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, data, demand_evaluator_index)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # pylint: disable=no-member
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    print_solution(data, routing, manager, assignment)


main()

Objective: 70508
Route for vehicle 0:
 0 Load(0) ->  32 Load(0) ->  1 Load(12) ->  22 Load(19) ->  28 Load(27) ->  31 Load(41) ->  8 Load(52) ->  26 Load(75) ->  7 Load(82) ->  23 Load(101) ->  43 Load(117) ->  24 Load(128) ->  14 Load(138) ->  0 Load(159)
Distance of the route: 15612m
Load of the route: 159

Route for vehicle 1:
 0 Load(0) ->  18 Load(0) ->  4 Load(41) ->  42 Load(50) ->  19 Load(63) ->  41 Load(72) ->  40 Load(99) ->  13 Load(106) ->  25 Load(129) ->  0 Load(157)
Distance of the route: 13172m
Load of the route: 157

Route for vehicle 2:
 0 Load(0) ->  46 Load(0) ->  38 Load(5) ->  9 Load(20) ->  50 Load(31) ->  30 Load(41) ->  34 Load(60) ->  21 Load(86) ->  35 Load(94) ->  36 Load(111) ->  29 Load(117) ->  16 Load(123) ->  11 Load(138) ->  0 Load(157)
Distance of the route: 14884m
Load of the route: 157

Route for vehicle 3:
 0 Load(0) ->  5 Load(0) ->  49 Load(21) ->  2 Load(39) ->  20 Load(69) ->  3 Load(97) ->  48 Load(113) ->  6 Load(130) ->  27 Load(145) ->  0 